In [ ]:
!pip install adaptkeybert datasets

In [9]:
# Add predictions to the DataFrame
df['argument_type'] = ['Claim' if pred == 1 else 'Premise' for pred in predictions]

In [10]:
df['stance'] = ['Positive' if pred == 2 else 'Negative' for pred in predictions]

In [ ]:
df

In [21]:
# Version 1.0
keywords_dict = {
    "Cost Management": [
        "efficiency ratio", "operating expenses", "cost control", "expense reduction",
        "margin improvement", "cost-cutting", "operational efficiency", "budget optimization","cost reduction"
    ],
    "Revenue Growth": [
        "top-line growth", "sales increase", "revenue streams",
        "organic growth", "customer acquisition", "growth in sales", "revenue drivers"
    ],
    "Cash Management": [
        "free cash flow", "liquidity", "cash reserves", "working capital", "capital allocation",
        "cash deployment", "treasury management", "cash optimization"
    ],
    "Market Position": [
        "competitive advantage", "industry leadership", "market dominance", "positioning",
        "peer comparison", "market share expansion","market share expansion","competitive landscape", "benchmarking"
    ],
    "Growth Strategy": [
        "strategic expansion", "scaling operations", "organic growth", "new markets",
        "business development", "innovation focus", "market penetration", "investment in growth"
    ],
    "Financial Performance": [
        "EBITDA", "net income", "earnings per share", "profit margins", "return on equity",
        "financial results", "performance metrics", "quarterly performance"
    ],
    "Capital Management": [
        "capital structure", "debt-to-equity ratio", "shareholder returns", "capital allocation",
        "fundraising", "equity issuance", "leverage", "capital efficiency"
    ],
    "Product Strategy": [
        "product innovation", "portfolio diversification", "pipeline", "product differentiation",
        "R&D investment", "product launch", "market offerings", "product development"
    ],
    "Supply Chain": [
        "logistics", "inventory management", "supply chain efficiency", "procurement",
        "distribution channels", "supply disruptions", "supplier relationships", "operational flow"
    ],
    "Regulatory Environment": [
        "compliance", "regulatory risk", "legal framework", "government policies",
        "regulation changes", "industry standards", "oversight", "policy impact"
    ],
    "Dividend Policy": [
        "dividend payout", "shareholder returns", "yield", "dividend increase", "dividend sustainability",
        "distribution", "payout ratio"
    ],
    "Mergers and Acquisitions": [
        "acquisition", "merger", "strategic fit", "synergies", "integration", "transaction",
        "buyout", "valuation"
    ],
    "Technology": [
        "digital transformation", "cloud computing", "AI", "innovation", "technology investment",
        "automation", "R&D", "platform development"
    ],
    "Customer Engagement": [
        "customer experience", "client relationships", "customer retention", "user satisfaction",
        "engagement metrics", "CRM", "customer feedback", "interaction"
    ],
    "Market Trends": [
        "industry trends", "market dynamics", "consumer behavior", "demand shifts",
        "economic indicators", "trend analysis", "emerging markets", "future outlook"
    ],
    "Investment Strategy": [
        "portfolio management", "asset allocation", "strategic investment", "growth investments",
        "capital deployment", "investment focus", "long-term strategy", "fund allocation"
    ],
    "Pricing Strategy": [
        "price optimization", "dynamic pricing", "competitive pricing",
        "pricing models", "price elasticity", "discount strategies", "profit maximization"
    ],
    "Geographic Performance": [
        "regional results", "international growth", "domestic performance", "market expansion",
        "geographic trends", "area-specific performance", "country operations", "global footprint"
    ],
    "Balance Sheet": [
        "assets", "liabilities", "equity", "debt management", "financial health", "liquidity",
        "book value", "capital structure"
    ],
    "Corporate Governance": [
        "board structure", "executive decisions", "shareholder rights", "governance framework",
        "transparency", "oversight", "ethics", "responsible management"
    ],
    "Operational Performance": [
        "efficiency", "productivity", "execution", "operational metrics", "process improvement",
        "cost-effectiveness", "operation scalability", "workflow"
    ],
    "Risk Management": [
        "risk assessment", "mitigation", "financial risk", "operational risk", "compliance risk",
        "market risk", "credit risk", "exposure"
    ],
    "Research and Development": [
        "R&D spend", "innovation", "product development", "research projects", "technology advances",
        "research focus", "development pipeline", "technical investment"
    ],
    "Market Performance": [
        "stock performance", "market capitalization", "share price", "index comparison",
        "market valuation", "trading volumes", "price movements", "investor confidence"
    ],
    "Growth Drivers": [
        "key drivers", "business catalysts", "market opportunities", "expansion plans",
        "strategic initiatives", "long-term growth", "accelerators", "revenue sources"
    ],
    "Financial Guidance": [
        "outlook", "projections", "forecasts", "earnings trajectory", "fiscal guidance",
        "profit expectations", "guidance updates", "targets"
    ],
    "Brand Strategy": [
        "brand positioning", "image", "market identity", "consumer perception",
        "branding efforts", "loyalty programs", "reputation", "brand value"
    ],
    "E-commerce": [
        "online sales", "digital marketplace", "e-retail", "omnichannel", "e-commerce growth",
        "platform usage", "internet sales", "mobile commerce"
    ],
    "Sustainability": [
        "green initiatives", "environmental impact", "ESG", "carbon footprint",
        "sustainable practices", "renewable energy", "social responsibility", "climate goals"
    ],
    "Shares Repurchase": [
        "buyback", "repurchase programs", "shareholder value", "equity reduction",
        "return to shareholders", "capital returns", "stock repurchase", "treasury stock"
    ],
    "Competitive Environment": [
        "competition", "market landscape", "industry dynamics", "rivals",
        "competitive positioning", "market pressures", "competitor analysis", "peer performance"
    ],
    "Operational Risks": [
        "systemic risks", "failure risks", "operational exposure", "business continuity",
        "crisis management", "technical risks", "process failure", "contingency planning"
    ],
    "Financial Health": [
        "balance sheet strength", "liquidity ratio", "financial stability", "creditworthiness",
        "solvency", "capital adequacy", "fiscal soundness", "financial robustness"
    ],
    "Sales Strategy": [
        "sales channels", "go-to-market strategy", "revenue generation", "customer acquisition",
        "sales force", "direct selling", "channel optimization", "distribution network"
    ],
    "Macroeconomic Impact": [
        "economic trends", "interest rates", "inflation", "GDP impact", "fiscal policy",
        "global economy", "macroeconomic shifts", "monetary policy"
    ],
    "Innovation": [
        "disruptive technology", "creativity", "tech-driven solutions", "product breakthroughs",
        "R&D focus", "innovative practices", "next-gen technology", "advanced solutions"
    ]
}


In [33]:
# Version 2.0 (Based on 1.0)

keywords_dict = {
    "Financial Efficiency": [
        "cost control", "expense reduction", "margin improvement", "budget optimization", 
        "efficiency ratio", "operational efficiency", "cost reduction", "expense management"
    ],
    "Revenue and Growth": [
        "top-line growth", "sales increase", "market share expansion", "revenue drivers",
        "organic growth", "growth strategy", "customer acquisition", "revenue generation"
    ],
    "Liquidity and Cash Management": [
        "free cash flow", "liquidity", "cash reserves", "working capital", "capital allocation",
        "treasury management", "cash optimization", "debt repayment"
    ],
    "Market Leadership": [
        "competitive advantage", "industry leadership", "market dominance", "peer comparison",
        "positioning", "benchmarking", "market share", "competitive landscape"
    ],
    "Product Innovation": [
        "product innovation", "portfolio diversification", "product differentiation", 
        "R&D investment", "new product launch", "product pipeline", "development pipeline"
    ],
    "Supply Chain and Logistics": [
        "logistics", "inventory management", "procurement", "supply chain efficiency",
        "distribution channels", "supplier relationships", "supply disruptions", "operational flow"
    ],
    "Compliance and Regulation": [
        "regulatory risk", "legal framework", "compliance", "government policies",
        "industry standards", "policy impact", "regulation changes", "oversight"
    ],
    "Investment and Capital Allocation": [
        "strategic investment", "capital deployment", "fundraising", "equity issuance",
        "debt-to-equity ratio", "capital structure", "shareholder returns", "leverage"
    ],
    "Customer Experience": [
        "customer engagement", "customer experience", "client relationships", "user satisfaction",
        "customer retention", "CRM", "interaction", "feedback"
    ],
    "Digital Transformation and Technology": [
        "digital transformation", "cloud computing", "automation", "platform development", 
        "AI", "technology investment", "innovation", "tech-driven solutions"
    ],
    "Environmental and Social Responsibility": [
        "sustainability", "ESG", "carbon footprint", "renewable energy", "climate goals",
        "social responsibility", "green initiatives", "environmental impact"
    ],
    "Risk and Operational Management": [
        "risk assessment", "financial risk", "operational risk", "compliance risk",
        "crisis management", "business continuity", "technical risks", "contingency planning"
    ],
    "Economic and Market Trends": [
        "economic trends", "consumer behavior", "market dynamics", "macroeconomic impact",
        "demand shifts", "emerging markets", "global economy", "inflation impact"
    ]
}


In [47]:
# Version 3.0 (Based on unsupervised list of LLaMA)
keywords_dict = {
    "Financial Performance": {
        "Financial Metrics": ["key metrics", "financial ratios", "performance indicators","Earnings per Share","EPS","earnings per share beat", "EPS outperformance", "above EPS expectations", "balance sheet", "cashflow","liquidity","working capital"],
        "Revenue": ["operational revenue", "business income", "operating profit","increased income","sales increase"],
        "Return On Capital": ["capital efficiency", "investment return", "capital ROI"],
        "Margins": ["EBITDA margins","operating margins","shrinking margins", "reduced profitability", "margin decrease","margin increase", "profit margins", "cost-to-revenue", "gross profit","profitability changes","profitability improvement","operational efficiency", "profit margin", "operating ratio"],
        "Cost Management": ["expense control", "cost reduction", "efficiency savings","expense trimming", "cost containment","reduced expenses","cost optimization"],
        "Efficiency Gains": ["productivity improvements", "efficiency boosts", "streamlined processes"],
    },
    "Company": {
        "Long-term Growth": ["sustainable growth", "long-term trajectory", "future growth","scalability", "large-scale expansion"],
        "Unit Growth": ["unit expansion", "product units", "unit volume"],
        "Mergers & Acquisition": ["business acquisition", "M&A activity", "buyout approach", "merging"],
        "Refranchising": ["franchise model", "franchise transitions", "refranchising plans"],
        "Sustainability Efforts": ["green initiatives", "environmental impact", "sustainable practices","ESG","SRI","sustainability","green energy adoption"],
    },
    "Product": {
        "Product Innovation": ["new features", "innovative products", "product advancements","new product","product portfoloio","product launch"],
        "Autonomous Technology": ["self-driving", "autonomous systems", "driverless innovation"],
        "Cloud Computing": ["cloud migration", "cloud-based services", "cloud platform"],
        "Production Disruptions": ["manufacturing delays", "supply issues", "production halts","factory issues","production stops","factory problems"],
    },
    "Market": {
        "Market Share": ["share of market", "industry share", "market proportion"],
        "Market Expansion": ["new markets", "geographical reach", "market entry","worldwide expansion","market trends", "sector growth"],
        "Competitors":["market rivalry", "competitive threats", "industry competition","market differentiation", "unique value", "competitive advantage"],
        "Global Presence": ["international footprint", "global operations", "worldwide coverage"],
		"Regulations":["tax regulations","regulatory risks","governmental influence","government incentives", "state funding", "public backing"],
        "Foreign Exchange Market": ["foreign exchange impact", "currency challenges", "FX pressures"],
        "Litigation Uncertainty": ["legal disputes", "lawsuit risks", "litigation challenges","lawsuits against","penalty fees","expected settlement"],
        "Partnerships and Collaborations":["strategic alliances","partner relationships","joint venture"],
        "Supply Chain":["logistics optimization", "supply logistics", "supply chain strategies","material shortage","supply constraints"],
        "Economic Conditions": ["economic environment", "market economy", "macroeconomic factors","recession","expansion"],
    },
    "Customer": {
        "Customer Engagement": ["user interaction", "engagement metrics", "customer involvement"],
        "Brand Loyalty": ["loyal customers", "repeat buyers", "customer retention","customer loyalty","frequent buyer"],
        "Enhanced User Experience": ["UX improvements", "better interface", "improved usability"],
        "Customer Behaviour": ["increased visitors", "traffic rise", "customer influx","customer visits","store traffic","purchasing trends", "spending habits", "customer expenditures","per-visit spending","ticket size"]
    },
    "Technology": {
        "Digital Transformation": ["digital shift", "IT modernization", "digital upgrades"],
        "AI": ["AI advantages", "machine learning impact", "AI-driven improvements","machine learning","deep learning","natural language processing","ai"],
        "Cybersecurity": ["data protection", "network security", "cyber risk"],
        "Automation": ["process automation", "automated systems", "robotics","RPA"]
    },
    "Leadership and Management": {
        "Strong Leadership": ["visionary management", "effective leadership", "executive strength"],
        "Management Belief": ["executive confidence", "leadership expectations", "management trust"],
        "Strategic Shift": ["business pivot", "organizational change", "strategic redirection"],
    }
}


In [48]:
from adaptkeybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Training

# Testing

In [49]:
# Initialize AdaptKeyBERT and SentenceTransformer
kw_model = KeyBERT()


# Flattening the keywords into a single list
seed_words = [
    argument 
    for category, arguments in keywords_dict.items() 
    for argument, keywords in arguments.items() 
    for keyword in keywords
]

sentence_transformer = SentenceTransformer('all-MiniLM-L6-v2')

def classify_sentence(sentence, kw_model, keywords_dict,seed_words, threshold=0.65):
    keywords = kw_model.extract_keywords(sentence, top_n=5,seed_keywords=seed_words,nr_candidates=50,)
    extracted_keywords = [kw[0] for kw in keywords]

    #print(f"sentence: {sentence}, Extracted_Keywords: {extracted_keywords}, keywords: {keywords}")

    results = {}
    for category, arguments in keywords_dict.items():
        argument_scores = {}
        for argument, seed_words in arguments.items():
            seed_embeddings = sentence_transformer.encode(seed_words)
            keyword_embeddings = sentence_transformer.encode(extracted_keywords)
            similarities = cosine_similarity(keyword_embeddings, seed_embeddings)
            max_similarity = similarities.max() if similarities.size > 0 else 0
            if max_similarity >= threshold:
                argument_scores[argument] = max_similarity
        
        if argument_scores:
            results[category] = argument_scores
    
    return results if results else {"Uncategorized": "No relevant category found"}


# Example sentences
sentences = [
    "The latest advancements in AI are groundbreaking.",
    "Apples Revenue grew year-to-year by 25%",
    "Eating vegetables is good for your health.",
    "This is a generic sentence with no clear category.",
    "The company was able to cut the costs for the whole year",
    "They acquired Monsato for a fair price which added more services to their portfolio",
    "Excellent results. Revenues of 2.39B ahead of cons. 2.35B while FY outlook gets a 100M boost. Billings 16 yy ahead of cons. 7 on a tough compare due to increasing but well understood seasonality 12 month target in the business. Management visibility sounds very good, supported by deferred revenue strength with total deferred plus backlog now standing at 14B. Operating cash flow of 1.2B was a record and 52 week range higher than all of F16. Operating margin expansion continues at a Dividend Yield Market Cap m balanced rate 125150 bps this FY, driving nice cash flow growth 2021 yy in F18 reaffirmed.",
    "SALESFORCE.COM, INC. CRM, Buy, 100.00 PT Current Price 87.75 Analyst Joel.Fishbein",
    "3. Gaining market share through supplychain dominance.",
    "As competitors grapple with chip shortages, Apples share of global smartphone sales breached 5year highs, hitting 21 in 4Q21.",
    "In the highend market, Samsung was hit by its Texas semiconductor fabrication plants monthlong power shutdown and Vietnam factory suspensions due to COVID19.",
    "Samsungs new Galaxy Notes launch in may be delayed as a result.",
    "Apple managed to mitigate shortages in 3QFY21, with the revenue impact coming in lower than its US34bn guidance.",
    "Apple holds buying power over suppliers through scale purchases and upfront cash payments, leaving competitors handling price shocks and shortages created by Apple deals."
]

# Test classification
for sentence in sentences:
    categories = classify_sentence(sentence, kw_model, keywords_dict,seed_words)
    print(f"Sentence: {sentence}\nCategories: {categories}\n")


Sentence: The latest advancements in AI are groundbreaking.
Categories: {'Product': {'Product Innovation': 0.78166187}, 'Technology': {'AI': 0.9999999}}

Sentence: Apples Revenue grew year-to-year by 25%
Categories: {'Financial Performance': {'Revenue': 0.79779124, 'Margins': 0.78512156}}

Sentence: Eating vegetables is good for your health.
Categories: {'Uncategorized': 'No relevant category found'}

Sentence: This is a generic sentence with no clear category.
Categories: {'Uncategorized': 'No relevant category found'}

Sentence: The company was able to cut the costs for the whole year
Categories: {'Financial Performance': {'Margins': 0.6727456, 'Cost Management': 0.6862726}}

Sentence: They acquired Monsato for a fair price which added more services to their portfolio
Categories: {'Company': {'Mergers & Acquisition': 0.6576241}}

Sentence: Excellent results. Revenues of 2.39B ahead of cons. 2.35B while FY outlook gets a 100M boost. Billings 16 yy ahead of cons. 7 on a tough compare d